<a href="https://colab.research.google.com/github/dchu1/AI_P2_cl/blob/master/SynapticIntelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
n_tasks = 20
n_epochs = 3
print_messages = False

# Imports

In [0]:
pip install ax-platform

     |████████████████████████████████| 501kB 4.7MB/s 
     |████████████████████████████████| 225kB 15.1MB/s 
     |████████████████████████████████| 235kB 19.6MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.0.1-py2.py3-none-any.whl size=390441 sha256=e680f242fca8fc58d6e46da845572fffc6096d6ff449b5c0253001c64e24568f
  Stored in directory: /root/.cache/pip/wheels/10/2f/7a/3328e5713d796daeec2ce8ded141d5f3837253fc3c2a5c62e0
Successfully built gpytorch


In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data as data_utils
import numpy as np
import subprocess
import os
import random
import matplotlib.pyplot as plt

from torch.nn.parameter import Parameter
from torch.nn import init
from torch.nn import Module
from torch.nn import init
from torchvision import datasets, transforms
from PIL import Image
from IPython.core.debugger import set_trace

n_tasks = 20
n_epochs = 3
print_messages = False

# Constructing Data Set

In [0]:
def rotate_dataset(d, rotation):
  result = torch.FloatTensor(d.size(0), 784)
  tensor = transforms.ToTensor()

  for i in range(d.size(0)):
    img = Image.fromarray(d[i].numpy(), mode="L")
    result[i] = tensor(img.rotate(rotation)).view(784)
  return result

mnist_path = "mnist.npz"
if not os.path.exists(os.path.join("/content", mnist_path)):
  subprocess.call("wget https://s3.amazonaws.com/img-datasets/mnist.npz", shell=True)

f = np.load(mnist_path)
x_tr = torch.from_numpy(f["x_train"])
y_tr = torch.from_numpy(f["y_train"]).long()
x_te = torch.from_numpy(f["x_test"])
y_te = torch.from_numpy(f["y_test"]).long()
f.close()

# Rotate Dataset
tasks_tr = []
tasks_te = []
mnist_rot_path = "mnist_rotations.pt"
if not os.path.exists(os.path.join("/content", mnist_rot_path)):
    torch.manual_seed(0)

    for t in range(n_tasks):
      min_rot = 1.0 * t / n_tasks * (180.0 - 0.0) + 0.0
      max_rot = 1.0 * (t + 1) / n_tasks * (180.0 - 0.0) + 0.0
      rot = random.random() * (max_rot - min_rot) + min_rot

      tasks_tr.append([rot, rotate_dataset(x_tr, rot), y_tr])
      tasks_te.append([rot, rotate_dataset(x_te, rot), y_te])

    torch.save([tasks_tr, tasks_te], 'mnist_rotations.pt')
else:
    tasks_tr, tasks_te = torch.load('/content/mnist_rotations.pt')

# Defining Synaptic Intelligence Model (Simple)

In [0]:
class SIModel(nn.Module):
    def __init__(self):
        super(SIModel, self).__init__()

        # SI Hyperparameters
        self.si_c = 0.           #-> hyperparam: how strong to weigh SI-loss ("regularisation strength")
        self.epsilon = 0.1      #-> dampening parameter: bounds 'omega' when squared parameter-change goes to 0
    
    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.kaiming_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def init(self, n_neurons):
        # Our Network
        self.net = nn.Sequential(
            nn.Linear(28*28, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, 10)
        )
        self.net.apply(self.init_weights)
        # self.fc1 = nn.Linear(28*28, n_neurons)
        # self.fc2 = nn.Linear(n_neurons, n_neurons)
        # self.fc3 = nn.Linear(n_neurons, 10)


    def forward(self, x):
        return self.net(x)

        # x = self.fc1(x)
        # x = F.relu(x)
        # x = self.fc2(x)
        # x = F.relu(x)
        # y = self.fc3(x)
        # return y

    def update_omega(self, W):
        '''After completing training on a task, update the per-parameter regularization strength.

        [W]         <dict> estimated parameter-specific contribution to changes in total loss of completed task
        '''

        # Loop over all parameters
        for n, p in self.named_parameters():
            if p.requires_grad:
                n = n.replace('.', '__')

                # Find/calculate new values for quadratic penalty on parameters
                p_prev = getattr(self, '{}_SI_prev_task'.format(n))
                p_current = p.detach().clone()
                p_change = p_current - p_prev
                
                omega_add = W[n]/(p_change**2 + self.epsilon)
                try:
                    omega = getattr(self, '{}_SI_omega'.format(n))
                except AttributeError:
                    omega = p.detach().clone().zero_()
                omega_new = omega + omega_add

                # Store these new values in the model
                self.register_buffer('{}_SI_prev_task'.format(n), p_current)
                self.register_buffer('{}_SI_omega'.format(n), omega_new)

    def surrogate_loss(self):
        '''Calculate SI's surrogate loss.'''
        try:
            losses = []
            for n, p in self.named_parameters():
                if p.requires_grad:
                    # Retrieve previous parameter values and their normalized path integral (i.e., omega)
                    n = n.replace('.', '__')
                    prev_values = getattr(self, '{}_SI_prev_task'.format(n))
                    omega = getattr(self, '{}_SI_omega'.format(n))
                    # Calculate SI's surrogate loss, sum over all parameters
                    losses.append((omega * (p-prev_values)**2).sum())
            return sum(losses)
        except AttributeError:
            # SI-loss is 0 if there is no stored omega yet
            return torch.tensor(0., device=self._device())

    def _device(self):
        return next(self.parameters()).device


# Running our experiment

In [0]:
def train_task(model, device, train_loader, optimizer, batch_log = 0):
    model.train()
    # Prepare <dicts> to store running importance estimates and param-values before update
    W = {}
    param_old = {}
    for name, param in model.named_parameters():
        if param.requires_grad:
            name = name.replace('.', '__')
            W[name] = param.data.clone().zero_()
            param_old[name] = param.data.clone()

    losses = []
    total_losses = []
    for k in range(n_epochs):
        if print_messages:
            print("----> Epoch {}:".format(k))
        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # Get the prediction
            y_hat = model(x)

            # Calculate training-precision
            precision = (y == y_hat.max(1)[1]).sum().item() / x.size(0)

            # Calculate the loss using cross entropy
            # and the surrogate loss
            loss = F.cross_entropy(input=y_hat, target=y, reduction='mean')
            surrogate_loss = model.surrogate_loss()
            total_loss = loss + model.si_c * surrogate_loss

            # Backpropagate errors
            total_loss.backward()

            # Take optimization-step
            optimizer.step()

            # Update running parameter importance estimates in W
            # "In practice, we can approximate w as the running sum of the 
            # product of the gradient g(w) and the parameter update" 
            for name, param in model.named_parameters():
                if param.requires_grad:
                    name = name.replace('.', '__')
                    if param.grad is not None:
                        W[name].add_(-param.grad*(param.detach()-param_old[name]))
                    param_old[name] = param.detach().clone()

            # Print out a log
            if batch_idx % batch_log == 0:
                losses.append(loss.item())
                total_losses.append(total_loss.item())
                if print_messages:
                    print('---->[{}/{} ({:.0f}%)]\tPrecision: {:.6f}\tLoss: {:.6f}\tSurrogate Loss: {:.6f}\tTotal Loss: {:.6f}'.format(
                        batch_idx * len(x), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), 
                        precision, loss.item(), surrogate_loss.item(), total_loss.item()))
            
    # After finishing training on a task, update the omega value in the model
    model.update_omega(W)

    return losses, total_losses
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            test_loss += F.cross_entropy(input=y_hat, target=y, reduction='mean')
            pred = y_hat.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    return correct, test_loss

def eval_on_tasks(model, device, test_loaders):
    acc = []
    test_losses = []
    for j in range(n_tasks):
        correct, test_loss = test(model, device, test_loaders[j])
        acc.append(100. * correct / len(test_loaders[j].dataset))
        test_losses.append(test_loss)
        if print_messages:
            print('---->Test set {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
                j, test_loss, correct, len(test_loaders[j].dataset),
                100. * correct / len(test_loaders[j].dataset)))
    return acc, test_losses

In [0]:
config={
    "lr": 0.003, 
    "si_c": 0.152, 
    "si_epsilon": 0.01,
    "optimizer": "adam",
    "batch_size": 64,
    "n_neurons": 100,
    "sample_size": 60000
    }

In [0]:
def main(config): 
    # Use cuda?
    cuda = torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")

    # Create Model
    model = SIModel()
    model.init(config["n_neurons"])
    model.to(device)
    optim_list = [{'params': filter(lambda p: p.requires_grad, model.parameters()), 'lr': config['lr']}]
    if config['optimizer'] == "adam":
        optimizer = optim.Adam(optim_list, betas=(0.9, 0.999))
    else:
        optimizer = optim.SGD(optim_list)

    # SI Parameters
    model.si_c = config["si_c"]
    model.epsilon = config["si_epsilon"]

    for name, param in model.named_parameters():
        if param.requires_grad:
            name = name.replace('.', '__')
            model.register_buffer('{}_SI_prev_task'.format(name), param.data.clone())

    # Load our test data
    test_loaders = []
    for i in range(n_tasks):
        test_loaders.append(data_utils.DataLoader(data_utils.TensorDataset(tasks_te[i][1], tasks_te[i][2]), batch_size=1000, shuffle = False))

    # Training
    if print_messages:
        print("--> Training:")

    total_acc = []
    total_test_losses = []
    
    # Before we start training we will get a baseline by evaluating our tasks
    acc, test_losses = eval_on_tasks(model, device, test_loaders)
    total_acc.append(acc)
    total_test_losses.append(test_losses)

    for i in range(n_tasks):
        if print_messages:
            print("--> Training Task {}:".format(i))

        perm = np.random.permutation(tasks_tr[i][1].size(0))
        perm = perm[:config['sample_size']]
        train_data = data_utils.TensorDataset(tasks_tr[i][1], tasks_tr[i][2])
        train_loader = data_utils.DataLoader(train_data, batch_size=config["batch_size"], 
                                      sampler = data_utils.SubsetRandomSampler(perm), drop_last = True)
        # train_loader = data_utils.DataLoader(train_data, batch_size=config["batch_size"], 
        #                                      shuffle = True, drop_last = True)
        
        train_losses, total_train_losses = train_task(model, device, train_loader, optimizer, 100)
        
        # Reset the optimizer (if using adam)
        if config['optimizer'] == "adam":
            model.optimizer = optim.Adam(optim_list, betas=(0.9, 0.999))

        if print_messages:
            print(train_losses)
            print(total_train_losses)
            print("--> Finished Training Task {}. Starting Test phase:".format(i))

        # Get our accuracy metrics on all test sets
        # acc = []
        # test_losses = []
        # for j in range(n_tasks):
        #     correct, test_loss = test(model, device, test_loaders[j])
        #     acc.append(100. * correct / len(test_loaders[j].dataset))
        #     test_losses.append(test_loss)
        #     if print_messages:
        #         print('---->Test set {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        #             j, test_loss, correct, len(test_loaders[j].dataset),
        #             100. * correct / len(test_loaders[j].dataset)))
        acc, test_losses = eval_on_tasks(model, device, test_loaders)
        total_acc.append(acc)
        total_test_losses.append(test_losses)
    
    return total_acc, total_test_losses

In [59]:
total_acc, total_test_losses = main(config)

# Get the accuracy metric as defined by Facebook paper: sum(R_Ti) 
# where T is the test set of the last Task and i is the current trained task
average_acc = np.mean(total_acc[n_tasks-1])
print("Accuracy:", average_acc)
print("Confusion matrix:")
print('\n'.join([','.join([str(item) for item in row]) for row in total_acc]))

Accuracy: 58.57899999999999
Confusion matrix:
12.46,13.32,13.42,11.23,12.26,13.76,13.83,13.41,11.72,9.1,9.02,10.44,10.15,10.06,11.12,10.45,10.26,9.74,8.68,8.73
97.02,91.66,90.99,77.71,64.69,38.91,30.18,17.74,13.59,11.06,11.46,10.93,13.55,13.83,18.22,21.61,24.21,25.25,26.3,27.73
95.07,95.9,95.67,89.43,81.09,54.05,42.73,24.81,19.4,15.1,13.3,12.02,11.42,11.24,14.24,18.85,22.66,25.54,27.37,29.33
95.39,95.8,95.61,90.81,83.16,55.96,43.44,25.85,20.0,15.65,13.88,12.51,12.45,12.17,16.0,21.23,25.25,27.56,28.96,30.67
92.93,95.71,95.68,92.9,87.94,63.83,51.01,30.51,23.44,17.28,15.48,13.61,12.63,12.46,15.79,20.42,24.2,26.91,28.18,30.38
88.18,93.68,93.93,93.84,92.1,78.17,68.8,47.21,35.42,25.49,19.51,15.91,12.33,12.16,13.54,18.49,21.92,24.76,27.03,29.61
82.65,89.79,90.58,92.93,92.95,86.71,81.05,63.8,50.32,36.99,26.43,20.46,13.29,12.92,12.75,15.97,18.86,21.57,23.9,27.27
77.33,86.66,87.13,90.71,91.14,89.82,87.19,76.18,64.11,48.4,33.53,27.27,17.29,16.79,13.75,14.6,15.72,17.59,20.35,23.65
71.84,82.49,83.1

# Tune Hyperparamters using Ax

In [0]:
def tune(config, objective):
    total_acc, total_loss = main(config)
    if objective == "accuracy":
        return np.mean(total_acc[n_tasks-1])
    elif objective == "loss":
        return np.mean(total_loss[n_tasks-1])
    else:
        return

from ax import optimize
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "lr",
            "type": "range",
            "bounds": [1e-4, 0.4], 
            "log_scale": True,
            "value_type": "float",
        },
        {  
            "name": "si_c",
            "type": "range",
            "bounds": [0.01, 0.5],
            "value_type": "float",
        },
        {  
            "name": "si_epsilon",
            "type": "fixed",
            "value": 0.01,
            "value_type": "float",
        },
        {  
            "name": "batch_size",
            "type": "fixed",
            "value": 64,
            "value_type": "int",
        },
        {  
            "name": "sample_size",
            "type": "fixed",
            "value": 10000,
            "value_type": "int",
        },
        {  
            "name": "n_neurons",
            "type": "fixed",
            "value": 100,
            "value_type": "int",
        },
        {  
            "name": "optimizer",
            "type": "fixed",
            "value": "adam",
            "value_type": "str",
        },
    ],
    evaluation_function=lambda p: tune(p, "accuracy"),
    objective_name='accuracy',
)
print(best_parameters)
print(values)
    #evaluation_function=lambda p: np.mean(main(p)[n_tasks-1]),
    #minimize=True,)

[INFO 04-05 02:39:10] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 8 arms, GPEI for subsequent arms], generated 0 arm(s) so far). Iterations after 8 will take longer to generate due to model-fitting.
[INFO 04-05 02:39:10] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-05 02:39:10] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-05 02:40:23] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-05 02:41:35] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-05 02:42:47] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-05 02:43:59] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-05 02:45:11] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-05 02:46:23] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-05 02:47:34] ax.service.managed_loop: Running optimization t

{'lr': 0.0029937139316947364, 'si_c': 0.15201809808880937, 'si_epsilon': 0.01, 'batch_size': 64, 'sample_size': 10000, 'n_neurons': 100, 'momentum': 0.9, 'optimizer': 'adam'}
({'accuracy': 58.60698750153793}, {'accuracy': {'accuracy': 2.0230659953751954e-05}})


In [43]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "lr",
            "type": "range",
            "bounds": [1e-4, 0.4], 
            "log_scale": True,
            "value_type": "float",
        },
        {  
            "name": "si_c",
            "type": "fixed",
            "value": 0.152,
            "value_type": "float",
        },
        {  
            "name": "si_epsilon",
            "type": "fixed",
            "value": 0.01,
            "value_type": "float",
        },
        {  
            "name": "batch_size",
            "type": "choice",
            "values": [64, 128, 256],
            "value_type": "int",
        },
        {  
            "name": "sample_size",
            "type": "choice",
            "values": [1000, 5000, 10000, 20000, 40000, 60000],
            "value_type": "int",
        },
        {  
            "name": "n_neurons",
            "type": "fixed",
            "value": 100,
            "value_type": "int",
        },
        {  
            "name": "optimizer",
            "type": "fixed",
            "value": "adam",
            "value_type": "str",
        },
    ],
    evaluation_function=lambda p: tune(p, "accuracy"),
    objective_name='accuracy',
)
print(best_parameters)
print(values)

[INFO 04-05 04:20:04] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-05 04:20:04] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-05 04:20:04] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-05 04:20:43] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-05 04:21:30] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-05 04:22:17] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-05 04:25:00] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-05 04:26:03] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-05 04:27:13] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-05 04:29:13] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-05 04:30:30] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-05 04:31:05] ax.service.managed_loop: Running optimization trial 10...
[INFO 04-05 04:35:11] ax.service.ma

{'lr': 0.0017180968355585689, 'batch_size': 64, 'sample_size': 60000, 'si_c': 0.152, 'si_epsilon': 0.01, 'n_neurons': 100, 'momentum': 0.9, 'optimizer': 'adam'}
({'accuracy': 59.4385}, {'accuracy': {'accuracy': 0.0}})
